# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [2]:
import json
import os

with open('../config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
import autogen
autogen.__version__

'0.9.1post0'

# Code Writing and Execution

In [ ]:
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

In [ ]:
code_writer_system_message = """
You are a highly capable AI coding agent within a multi-agent scientific system. Your role is to write complete, correct, and fully executable Python code to fulfill a given task **without requiring any human intervention**.

Output Format:
Always respond with the following structure:

Description:
<Concise explanation of what the code does, its inputs, and outputs.>

```python
<Complete, well-organized Python code>
```

Notes:
- Save all generated plots or output files and report their filenames.
- Never ask the user to manually modify the code.
"""

In [ ]:
code_writer_agent = ConversableAgent(
    name="code_writer_agent",
    system_message=code_writer_system_message.format(
        technical_query_history = assistant_coder.context_variables.get('technical_query_history'),
        plan = assistant_coder.context_variables.get('plan')
    ),
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution for this agent.
)


# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=1000,  # Timeout for each code execution in seconds.
    work_dir="./code_dir",  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    is_termination_msg=lambda x: "TERMINATE" in (x.get("content", "") or "").upper(),
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=task,
    summary_method="reflection_with_llm",
    summary_args = {
    "summary_prompt": (
        "Provide a complete summary of the code execution. "
        "Indicate whether the code was executed successfully or failed, and include the reason if it failed. "
        "If successful, list all results generated by the code, such as numerical outputs, plots (with their names), file names, and any other return values. "
        "Ensure the user is fully informed about all execution outcomes."
    )
    },
    max_rounds=10,
)